In [ ]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import prepare,analyse,constructBasicDict,tailorData

### Hyper parameters setting

In [ ]:
hparams = {
    'npratio':4,
    'mode':'train',
    'scale':'demo',
    'batch_size':10,
    'his_size':50,
    'title_size':15,
    'device':'cuda:1',
    'attrs': ['title'],
    'news_id':True,
    'k': 0,
    'validate':False
}
# torch.cuda.set_device(device)

### Construct necessary dictionaries
- already done and the results are in `data/dictionaries`

In [ ]:
constructBasicDict(attrs=['title'],path='/home/peitian_zhang/Data/MIND')

### View data

In [ ]:
vocab, loaders = prepare(hparams)

In [ ]:
# loader_train
a = next(iter(loaders[0]))
# loader_dev
b = next(iter(loaders[1]))

In [ ]:
a['candidate_title_pad'].shape,b['candidate_title_pad'].shape

### Tailor Data to demo size

In [ ]:
# tailor 2000 impressions from MINDsmall_train to form MINDdemo_train
tailorData('/home/peitian_zhang/Data/MIND/MINDsmall_train/behaviors.tsv',2000)

tailorData('/home/peitian_zhang/Data/MIND/MINDsmall_dev/behaviors.tsv',500)

### Analyze MIND Datasets
- average title length
- average abstract length
- average history length
- average impression capacity
- count of history exceeding 50
- count of empty history
- count of multi-clicked impressions 

In [ ]:
analyse(hparams)

## The rest is developing

In [ ]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import prepare,analyse,constructBasicDict,tailorData
from utils.MIND import MIND
import numpy as np
import re
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, IterableDataset
from utils.utils import newsample, getId2idx, word_tokenize_vocab, getVocab, my_collate, worker_init_fn
from torch.utils.data import DataLoader

In [ ]:
hparams = {
    'npratio':4,
    'mode':'train',
    'scale':'demo',
    'batch_size':10,
    'his_size':50,
    'title_size':15,
    'device':'cuda:1',
    'attrs': ['title'],
    'news_id':True,
    'k': 0,
    'validate':False,
    'bert':'bert-base-uncased'
}
# torch.cuda.set_device(device)

In [ ]:
mind = MIND(hparams, '/home/peitian_zhang/Data/MIND/MINDdemo_dev/news.tsv', '/home/peitian_zhang/Data/MIND/MINDdemo_dev/behaviors.tsv')

loader_train = DataLoader(mind, batch_size=hparams['batch_size'], pin_memory=True,
                                  num_workers=0, drop_last=False, collate_fn=my_collate, worker_init_fn=worker_init_fn)

In [ ]:
record = next(mind.__iter__())
record

In [ ]:
record['candidate_title'].shape, record['candidate_title_pad'].shape, record['clicked_title'].shape, record['clicked_title_pad'].shape

In [ ]:
next(iter(loader_train))